<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="100">
</p>

<h1>Curso Ingeniería de Características</h1>

<h3>Introducción a GX Core</h3>

<p> Julio Waissman Vilanova </p>

<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/ing-caract/blob/main/ejemplos/calidad/great_expectations_intro.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;"  width="30" /> Ejecuta en Colab</a>

</center>

En esta libreta hago una mezcla de la explicación genñerica de la estructura de [*GX Core Overview*](https://docs.greatexpectations.io/docs/core/introduction/gx_overview) con [*Try GX Core*](https://docs.greatexpectations.io/docs/core/introduction/try_gx) con mñinios ajustes para que se puedan ejecutar en *Colaboratory*. 

# GX Core overview

This overview is for new users of GX Core and those looking for an improved understanding of GX Core components and primary workflows. It is an ideal place to start before exploring more advanced topics found in the GX Core documentation.

## GX Core components and workflows

**Great Expectations** (GX) is a framework for describing data using expressive tests and then validating that the data meets test criteria. **GX Core** is a Python library that provides a programmatic interface to building and running data validation workflows using GX.

**GX Core** is versatile and supports a variety of workflows. It can be used for interactive, exploratory data validation as well as data validation within production deployments.

**GX components** are Python classes that represent your data and data validation entities.

**GX workflows** are programmatically defined data validation processes. GX workflows are built using GX components.

## The pattern of a GX workflow

All GX workflows share a common pattern:

1. Set up a GX environment
2. Connect to data
3. Define Expectations
4. Run Validations

At each workflow step, different GX components are defined and used. This section introduces the key GX Core components required to create a data validation workflow.

![](https://docs.greatexpectations.io/assets/images/gx_workflow_steps_and_components-a1511e31cccadb7158f1e8665cc90628.png)

In [ ]:
!pip install great_expectations

In [ ]:
import great_expectations as gx

print(gx.__version__)

## Set up a GX environment

A **Data Context** manages the settings and metadata for a GX workflow. In GX Core, the Data Context is a Python object that serves as the entrypoint for the GX Python API. You use the Data Context to define and run a GX workflow; the Data Context provides access to the configurations, metadata, and actions of your GX workflow components and the results of data validations.

All GX workflows start with the creation of a Data Context. For more information on the types of Data Context, see [Create a Data Context](https://docs.greatexpectations.io/docs/core/set_up_a_gx_environment/create_a_data_context).


In [ ]:
context = gx.get_context()


## Connect to data

A **Data Source** is the GX representation of a data store. The Data Source tells GX how to connect to your data, and supports connection to different types of data stores, including databases, schemas, and data files in cloud object storage.

A **Data Asset** is a collection of records within a Data Source. A useful analogy is: if a Data Source is a relational database, then a Data Asset is a table within that database, or the results of a select query on a table within that database.

A **Batch Definition** tells GX how to organize the records within a **Data Asset**. The Batch Definition Python object enables you to retrieve a **Batch**, or collection of records from a Data Asset, for validation at runtime. A Data Asset can be validated as a single Batch, or partitioned into multiple Batches for separate validations.

For more information on connecting to data, see [Connect to data](https://docs.greatexpectations.io/docs/core/connect_to_data/).



In [ ]:
# Import sample data into Pandas DataFrame.
import pandas as pd
df = pd.read_csv(
    "https://raw.githubusercontent.com/great-expectations/gx_tutorials/main/data/yellow_tripdata_sample_2019-01.csv"
)
df

In [ ]:
# Connect to data.
# Create Data Source, Data Asset, Batch Definition, and Batch.

data_source = context.data_sources.add_pandas("pandas")

data_asset = data_source.add_dataframe_asset(name="pd dataframe asset")

batch_definition = data_asset.add_batch_definition_whole_dataframe("batch definition")

batch = batch_definition.get_batch(batch_parameters={"dataframe": df})

## Define Expectations

An **Expectation** is a verifiable assertion about data. Similar to assertions in traditional Python unit tests, Expectations provide a flexible, declarative language for describing expected data qualities. An Expectation can be used to validate a Batch of data.

For a full list of available Expectations, see the [Expectation Gallery](https://greatexpectations.io/expectations/).

An **Expectation Suite** is a collection of Expectations. Expectation Suites can be used to validate a Batch of data using multiple Expectations, streamlining the validation process. You can define multiple Expectation Suites for the same data to cover different use cases, and you can apply the same Expectation Suite to different Batches.

For more information defining Expectations and creating Expectation Suites, see [Define Expectations](https://docs.greatexpectations.io/docs/core/define_expectations/).



In [ ]:
# Create Expectation Suite
suite = context.suites.add(
    gx.core.expectation_suite.ExpectationSuite(name="expectations")
)

In [ ]:
# Add two Expectations to the Suite
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeBetween(
        column="passenger_count",
        min_value=1,
        max_value=6
    )
)

suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeBetween(
        column="fare_amount",
        min_value=0
    )
)

suite

## Run Validations

A **Validation Definition** explicitly associates a Batch Definition to an Expectation Suite, defining what data should be validated against which Expectations.

A **Validation Result** is returned by GX after data validation. The Validation Results tell you how your data corresponds to what you expected of it.

A **Checkpoint** is the primary means for validating data in a production deployment of GX. Checkpoints enable you to run a list of Validation Definitions with shared parameters. Checkpoints can be configured to run **Actions**, and can pass Validation Results to a list of predefined Actions for processing.

**Actions** provide a mechanism to integrate Checkpoints into your data pipeline infrastructure by automatically processing Validation Results. Typical use cases include sending email alerts, Slack messages, or custom notifications based on the result of data validation.

**Data Docs** are human-readable documentation generated by GX that host your Expectation Suite definitions and Validation Results. Using Checkpoints and Actions, you can configure your GX workflow to automatically write Validation Results to a chosen Data Docs site.

For more information on defining and running Validations, see [Run Validations](https://docs.greatexpectations.io/docs/core/run_validations/).

In [ ]:
# Create Validation Definition.
validation_definition = context.validation_definitions.add(
    gx.core.validation_definition.ValidationDefinition(
        name="validation definition",
        data=batch_definition,
        suite=suite,
    )
)

In [ ]:
# Create Checkpoint
checkpoint = context.checkpoints.add(
    gx.checkpoint.checkpoint.Checkpoint(
        name="checkpoint", validation_definitions=[validation_definition]
    )
)

In [ ]:
# Run Checkpoint, and capture result.

checkpoint_result = checkpoint.run(batch_parameters={"dataframe": df})
print(checkpoint_result.describe())

## Customize GX Core workflows

While all GX Core workflows follow a shared pattern, the outcome and operation of a workflow can be customized based on how you create Batches, define Expectations, and run Validations. GX Core components are building blocks that can be applied in a variety of ways to satisfy your data validation use case.

For instance, a GX Core workflow might:

- Create a Batch using data from a Spark DataFrame and allow you to interactively validate the Batch with Expectations and immediately review the Validation Results. This workflow could serve to inform your exploration of which Expectations you want to use in a production deployment of GX.

- Connect to data in a SQL table, define multiple Expectation Suites that each test for a desired data quality characteristic, and use a Checkpoint to run all Expectation Suites. This workflow, when integrated with and triggered by an orchestrator, could enable automated, scheduled data quality testing on an essential data table.

- Connect to a group of SQL tables and define a collection of Data Assets, each batched on a time-based column, and validate the data within each Data Asset using the same Expectation Suite. This workflow could provide a way to implement consistent data quality testing across a sharded data infrastructure.

Equipped with an understanding of the GX Core components, you can design data validation workflows that logically and effectively validate your data across a variety of data store types, environments, and business use cases.



## Next steps

Go to the [Expectations Gallery](https://greatexpectations.io/expectations) and experiment with other Expectations.

If you're ready to start using GX Core with your own data, the [Set up a GX environment documentation](https://docs.greatexpectations.io/docs/core/installation_and_setup/install_gx) provides a more comprehensive guide to setting up GX to work with specific data formats and environments.

Check out [GX Cloud](https://greatexpectations.io/cloud). Sign up here and you could be validating your data in minutes. We also offer regular GX Cloud workshops: [click here to get more information and register](https://pages.greatexpectations.io/gx-cloud-workshops).